In [7]:
import math

# 矩阵乘法
def mat_mul(A, B):
    m, n = len(A), len(B[0])
    result = [[0] * n for _ in range(m)]
    for i in range(m):
        for j in range(n):
            result[i][j] = sum(A[i][k] * B[k][j] for k in range(len(B)))
    return result

# 矩阵转置
def transpose(A):
    return [[A[j][i] for j in range(len(A))] for i in range(len(A[0]))]

# 生成单位矩阵
def identity_matrix(n):
    return [[1 if i == j else 0 for j in range(n)] for i in range(n)]

# Householder变换
import numpy as np

def householder(A):
    """
    将输入的对称矩阵A转换为三对角对称矩阵
    使用Householder变换。
    """
    # 确保矩阵A是对称矩阵
    if not np.allclose(A, A.T):
        raise ValueError("输入矩阵A必须是对称矩阵")
    
    # 获取矩阵的大小
    n = A.shape[0]
    
    # 初始化变换矩阵P（将最终的三对角矩阵存储在A中）
    for k in range(n - 2):  # 只需要进行n-2次变换
        # 构造Householder向量
        x = A[k+1:n, k]  # 取矩阵A中下三角的一个列向量
        e1 = np.zeros_like(x)
        e1[0] = np.linalg.norm(x)
        
        # 计算向量v
        v = x + np.sign(x[0]) * e1
        v = v / np.linalg.norm(v)  # 单位化向量v
        
        # 计算Householder变换矩阵H
        H = np.eye(n)
        H[k+1:n, k+1:n] -= 2 * np.outer(v, v)
        
        # 更新A
        A = H @ A @ H.T
    
    return A

# QR分解（基于Gram-Schmidt正交化）
def qr_decomposition(A):
    n = len(A)
    Q = identity_matrix(n)
    R = [[0] * n for _ in range(n)]
    for j in range(n):
        v = [A[i][j] for i in range(n)]
        for i in range(j):
            R[i][j] = sum(Q[k][i] * A[k][j] for k in range(n))
            v = [v[k] - R[i][j] * Q[k][i] for k in range(n)]
        R[j][j] = math.sqrt(sum(vi ** 2 for vi in v))
        for i in range(n):
            Q[i][j] = v[i] / R[j][j]
    return Q, R

# QR算法求特征值
def qr_eigenvalues(A, tol=1e-10, max_iter=1000):
    Ak = A
    for _ in range(max_iter):
        Q, R = qr_decomposition(Ak)
        Ak = mat_mul(R, Q)
        off_diag = sum(abs(Ak[i][j]) for i in range(len(A)) for j in range(len(A)) if i != j)
        if off_diag < tol:
            break
    return [Ak[i][i] for i in range(len(A))]

# 定义矩阵 A
A = np.array([
    [52, 30, 49, 28],
    [30, 50, 8, 44],
    [49, 8, 46, 16],
    [28, 44, 16, 22]
],dtype=float)

# Householder变换使 A 变为三对角矩阵
tri_A = householder(A)

# 使用QR分解计算特征值
eigenvalues = qr_eigenvalues(tri_A)

# 统计正特征值和负特征值的数量
positive_count = sum(1 for ev in eigenvalues if ev > 0)
negative_count = sum(1 for ev in eigenvalues if ev < 0)

# 输出结果
print("特征值:", eigenvalues)
print("正特征值个数:", positive_count)
print("负特征值个数:", negative_count)

特征值: [132.6278753342083, 52.44229999841117, -11.541130784673522, -3.529044547946003]
正特征值个数: 2
负特征值个数: 2
